## Imports

In [ ]:
import json
from openai import OpenAI
from tqdm import tqdm
from collections import Counter
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction, sentence_bleu
import pandas as pd

## Load dataset

### Load from json

In [98]:
import json
import os

def is_inside_frame(text, frame):
    text_center_x = text["x"] + text["w"] / 2
    text_center_y = text["y"] + text["h"] / 2
    return (
        frame["x"] <= text_center_x <= frame["x"] + frame["w"] and
        frame["y"] <= text_center_y <= frame["y"] + frame["h"]
    )

# Load the JSON data
with open(r".\open-mantra-dataset\annotation.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Extract the testing sets
dataset_bubble = []
dataset_frame = []
dataset_page = []

for book in data:
    book_title = book["book_title"]
    for page in book["pages"]:
        page_index = page["page_index"]
        page_path = os.path.join("open-mantra-dataset", page["image_paths"]["ja"])
        page_text_ja = []
        page_text_en = []

        for text in page["text"]:
            if "text_ja" in text and "text_en" in text:
                dataset_bubble.append({
                    "text_ja": f"{text['text_ja']}",
                    "text_en": f"{text['text_en']}"
                })

        for frame_index, frame in enumerate(page["frame"]):
            frame_path = os.path.join("open-mantra-dataset_frames", "images", book_title, "ja", "frames", f"{(page_index-1):03d}_frame_{frame_index}.jpg")
            frame_text_ja = []
            frame_text_en = []

            for text in page["text"]:
                if is_inside_frame(text, frame):
                    frame_text_ja.append(f"{text['text_ja']}")
                    frame_text_en.append(f"{text['text_en']}")

            if frame_text_ja and frame_text_en:
                dataset_frame.append({
                    "text_ja": "\n".join(frame_text_ja),
                    "text_en": "\n".join(frame_text_en),
                    "frame_path": frame_path
                })
                page_text_ja.append("\n".join(frame_text_ja))
                page_text_en.append("\n".join(frame_text_en))
            else:
                dataset_frame.append({
                    "text_ja": "",
                    "text_en": "",
                    "frame_path": frame_path
                })

        if any(page_text_ja) and any(page_text_en):
            dataset_page.append({
                "text_ja": "\n".join(page_text_ja),
                "text_en": "\n".join(page_text_en),
                "page_path": page_path
            })
        else:
            dataset_page.append({
                "text_ja": "",
                "text_en": "",
                "page_path": page_path
            })

### Visualize dataset line by line (no context)

In [99]:
def visualize_bubble_dataset(dataset, limit=5):
    print(f"Line by line Dataset: {len(dataset)} lines")
    for i, item in enumerate(dataset[:limit], 1):
        print(f"* Liine {i}:")
        print(f"   * Japanese: {item['text_ja']}")
        print(f"   * English: {item['text_en']}")
        print()

visualize_bubble_dataset(dataset_bubble)

Line by line Dataset: 1592 lines
* Liine 1:
   * Japanese: 綴じ眼のシオラ
   * English: bound eye siora

* Liine 2:
   * Japanese: 朽鷹みつき
   * English: Mitsuki Kuchitaka

* Liine 3:
   * Japanese: だからっ
   * English: I'm telling you!!

* Liine 4:
   * Japanese: 知らないって言ってるだろっ
   * English: I don't know what you're talking about!

* Liine 5:
   * Japanese: そんな借金なんて!
   * English: i don't owe you!



### Visualize dataset by frame-level context

In [102]:
def visualize_frame_dataset(dataset, limit=5):
    print(f"Frame Dataset: {len(dataset)} frames")
    for i, item in enumerate(dataset[:limit], 1):
        print(f"* Frame {i}:")
        print(f"   * Image path: {item['frame_path']}")
        if item['text_ja'] and item['text_en']:
            bubbles = zip(item['text_ja'].split('\n'), item['text_en'].split('\n'))
            for j, (text_ja, text_en) in enumerate(bubbles, 1):
                print(f"   * Line {j}:")
                print(f"      * Japanese: {text_ja}")
                print(f"      * English: {text_en}")
        else:
            print("   * Empty frame")
        print()

visualize_frame_dataset(dataset_frame, limit=4)



Frame Dataset: 1069 frames
* Frame 1:
   * Image path: open-mantra-dataset_frames\images\tojime_no_siora\ja\frames\000_frame_0.jpg
   * Line 1:
      * Japanese: 綴じ眼のシオラ
      * English: bound eye siora
   * Line 2:
      * Japanese: 朽鷹みつき
      * English: Mitsuki Kuchitaka

* Frame 2:
   * Image path: open-mantra-dataset_frames\images\tojime_no_siora\ja\frames\001_frame_0.jpg
   * Empty frame

* Frame 3:
   * Image path: open-mantra-dataset_frames\images\tojime_no_siora\ja\frames\001_frame_1.jpg
   * Line 1:
      * Japanese: だからっ
      * English: I'm telling you!!

* Frame 4:
   * Image path: open-mantra-dataset_frames\images\tojime_no_siora\ja\frames\001_frame_2.jpg
   * Line 1:
      * Japanese: 知らないって言ってるだろっ
      * English: I don't know what you're talking about!
   * Line 2:
      * Japanese: そんな借金なんて!
      * English: i don't owe you!



### Visualize dataset by page-level context

In [103]:
def visualize_page_dataset(dataset, limit=3):
    print(f"Page Dataset: {len(dataset)} pages")
    for i, item in enumerate(dataset[:limit], 1):
        print(f"* Page {i}:")
        print(f"   * Image path: {item['page_path']}")
        if item['text_ja'] and item['text_en']:
            bubbles = zip(item['text_ja'].split('\n'), item['text_en'].split('\n'))
            for k, (bubble_ja, bubble_en) in enumerate(bubbles, 1):
                print(f"      * Line {k}:")
                print(f"         * Japanese: {bubble_ja}")
                print(f"         * English: {bubble_en}")
        else:
            print("   * Empty page")
        print()
visualize_page_dataset(dataset_page, limit=2)

Page Dataset: 214 pages
* Page 1:
   * Image path: open-mantra-dataset\images/tojime_no_siora/ja/000.jpg
      * Line 1:
         * Japanese: 綴じ眼のシオラ
         * English: bound eye siora
      * Line 2:
         * Japanese: 朽鷹みつき
         * English: Mitsuki Kuchitaka

* Page 2:
   * Image path: open-mantra-dataset\images/tojime_no_siora/ja/001.jpg
      * Line 1:
         * Japanese: だからっ
         * English: I'm telling you!!
      * Line 2:
         * Japanese: 知らないって言ってるだろっ
         * English: I don't know what you're talking about!
      * Line 3:
         * Japanese: そんな借金なんて!
         * English: i don't owe you!
      * Line 4:
         * Japanese: そうは言ってもなぁ
         * English: well, I'm sorry...
      * Line 5:
         * Japanese: レーネ...
         * English: lene...



## Define basic functions

In [61]:
from bert_score import score
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize
from nltk.translate import meteor_score
from nltk import word_tokenize

import base64
import requests

import warnings
warnings.filterwarnings("ignore", message="A module that was compiled using NumPy 1.x")
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

# Ensure NLTK data is downloaded
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('omw-1.4')

def ask_question(question, client, max_tokens=200, temperature=0, top_p=0.9):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                'role': 'user',
                'content': question,
            }
        ],
        model='llama3',
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        stream=False
    )
    answer = chat_completion.choices[0].message.content
    return answer

def process_image_llava(image_path, question, mode):
    api_url = "http://localhost:11434/api/generate"
    with open(image_path, "rb") as image_file:
        image_data = base64.b64encode(image_file.read()).decode("utf-8")
    
    options = {
        "num_keep": 5,
        "seed": 42,
        "top_k": 20,
        "top_p": 0.9,
        "tfs_z": 0.5,
        "temperature": 0.3,
        "repeat_penalty": 1.2,
        "presence_penalty": 1.5,
        "frequency_penalty": 1.0,
        "penalize_newline": True,
    }
    
    if mode == "reduced":
        options["num_predict"] = 25
    else:
        options["num_predict"] = 75
    payload = {
        "model": "llava",
        "prompt": question,
        "images": [image_data],
        "stream": False,
        "options": options
    }
    response = requests.post(api_url, json=payload)
    if response.status_code == 200:
        result = response.json()
        answer = result["response"]
        return answer
    else:
        raise Exception(f"API request failed with status code {response.status_code}")
    
def calculate_bleu(predictions, references):
    reference_corpus = [[ref.split()] for ref in references]
    prediction_corpus = [pred.split() for pred in predictions]
    #print("Reference", reference_corpus)
    #print("Prediction", prediction_corpus)
    # Create a smoothing function
    smoothie = SmoothingFunction().method4
    bleu4 = corpus_bleu(reference_corpus, prediction_corpus, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
    return bleu4

def calculate_bleu_tokenized(predictions, references):
    reference_corpus = [[word_tokenize(ref)] for ref in references]
    prediction_corpus = [word_tokenize(pred) for pred in predictions]
    
    smoothie = SmoothingFunction().method4
    bleu4 = corpus_bleu(reference_corpus, prediction_corpus, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
    return bleu4

def calculate_bertscore(predictions, references, lang="en"):
    P, R, F1 = score(predictions, references, lang=lang, verbose=False, device='cuda')
    
    return {
        "precision": P.mean().item(),
        "recall": R.mean().item(),
        "f1": F1.mean().item()
    }

def calculate_meteor(predictions, references):
    meteor_scores = []
    
    for pred, ref in zip(predictions, references):
        # Tokenize the sentences
        pred_tokens = word_tokenize(pred.lower())
        ref_tokens = word_tokenize(ref.lower())
        
        # Calculate METEOR score for this pair
        score = meteor_score.meteor_score([ref_tokens], pred_tokens)
        meteor_scores.append(score)
    
    # Return the average METEOR score
    return sum(meteor_scores) / len(meteor_scores)

def calculate_metrics(predictions, references):
    # Remove rows with empty predictions
    filtered_predictions = []
    filtered_references = []
    for pred, ref in zip(predictions, references):
        if pred.strip():  # Check if the prediction is not empty
            filtered_predictions.append(pred)
            filtered_references.append(ref)
    
    predictions = filtered_predictions
    references = filtered_references
    bertscore_results = calculate_bertscore(predictions, references)
    print(f"BERTScore: {bertscore_results}")

    bleu_results = calculate_bleu(predictions, references)
    print(f"BLEU4: {bleu_results}")

    bleu_token_results = calculate_bleu_tokenized(predictions, references)
    print(f"BLEU4 tokenized: {bleu_token_results}")

    meteor_score = calculate_meteor(predictions, references)
    print(f"METEOR Score: {meteor_score}")

## Model evaluation functions

### Unimodal

#### No context

In [ ]:

def evaluate_model_unimodal_bubbles(client, test_dataset, prompt):
    data = []
    references = []
    for example in tqdm(test_dataset, desc="Evaluating"):
        sentence = example['text_ja']
        ground_truth = example['text_en']
        predicted_translation = translate_llm_bubbles(sentence, prompt, client)
        data.append({
            'sentence': sentence,
            'ground_truth': ground_truth,
            'predicted_translation': predicted_translation
        })
        references.append(ground_truth)
    
    df = pd.DataFrame(data)
    return df

def translate_llm_bubbles(sentence, prompt, client):
    question = prompt.format(text=sentence)
    answer = ask_question(question, client)
    # Split the translations into sentences
    sentences = answer.split("\n")
    
    # Initialize an empty list to store the translated sentences
    translated_sentences = []
    
    # Iterate over each sentence
    for sentence in sentences:
        # Check if the sentence starts with a number followed by a dot and a space
        if sentence.strip() and sentence.split(".", 1)[0].isdigit():
            # Remove the number and dot from the beginning of the sentence
            sentence = sentence.split(".", 1)[1].strip()
        
            # Append the processed sentence to the list of translated sentences
            translated_sentences.append(sentence)
    
    # Join the translated sentences into a single string
    translated_text = "\n".join(translated_sentences)
    
    # Check if the translated_text is empty
    if not translated_text.strip():
        # If empty, append the last sentence as is
        translated_text = sentences[-1].strip()
    return translated_text

#### Frame level

In [ ]:
def evaluate_model_unimodal_frames(client, test_dataset, prompt):
    data = []
    
    for example in tqdm(test_dataset, desc="Evaluating"):
        frame_path = example['frame_path']
        bubbles_ja = example['text_ja'].split('\n')
        bubbles_en = example['text_en'].split('\n')
        
        if not bubbles_ja or all(not bubble.strip() for bubble in bubbles_ja):
            #print("*Empty frame*")
            continue
        
        bubbles_input = '\n'.join([f"{i+1}. {bubble}" for i, bubble in enumerate(bubbles_ja)])
        
        predicted_translation = translate_llm_frames(bubbles_input, prompt, client)
        predicted_lines = predicted_translation.split('\n')
        
        for line_ja, line_en, line_predicted in zip(bubbles_ja, bubbles_en, predicted_lines):
            data.append({
                'frame_path': frame_path,
                'bubble_ja': line_ja,
                'bubble_en': line_en,
                'predicted_translation': line_predicted
            })

    df = pd.DataFrame(data)
    return df

def translate_llm_frames(bubbles_input, prompt, client):
    #print("--------Text input: ---------\n", bubbles_input)
    question = prompt.format(text=bubbles_input)
    #print("--------Prompt: ---------\n", question)
    answer = ask_question(question, client)
    #print("--------Raw answer: ---------\n", answer)

    # Split the translations into sentences
    sentences = answer.split("\n")
    
    # Initialize an empty list to store the translated sentences
    translated_sentences = []
    
    # Iterate over each sentence
    for sentence in sentences:
        # Check if the sentence starts with a number followed by a dot and a space
        if sentence.strip() and sentence.split(". ", 1)[0].isdigit():
            # Remove the number and dot from the beginning of the sentence
            sentence = sentence.split(". ", 1)[1].strip()
        
            # Append the processed sentence to the list of translated sentences
            translated_sentences.append(sentence)
    
    # Join the translated sentences into a single string
    translated_text = "\n".join(translated_sentences)

    # Check if the translated_text is empty
    if not translated_text.strip():
        # If empty, append the last sentence as is
        translated_text = sentences[-1].strip()
    
    #print("--------Final answer: ---------")
    #print(translated_text)

    return translated_text

#### Page level

In [ ]:
def evaluate_model_unimodal_pages(client, test_dataset, prompt):
    data = []
    
    for example in tqdm(test_dataset, desc="Evaluating"):
        frame_path = example['page_path']
        bubbles_ja = example['text_ja'].split('\n')
        bubbles_en = example['text_en'].split('\n')
        
        if not bubbles_ja or all(not bubble.strip() for bubble in bubbles_ja):
            #print("*Empty page]*")
            continue
        
        bubbles_input = '\n'.join([f"{i+1}. {bubble}" for i, bubble in enumerate(bubbles_ja)])
        
        predicted_translation = translate_llm_pages(bubbles_input, prompt, client)
        predicted_lines = predicted_translation.split('\n')
        
        for line_ja, line_en, line_predicted in zip(bubbles_ja, bubbles_en, predicted_lines):
            data.append({
                'page_path': frame_path,
                'bubble_ja': line_ja,
                'bubble_en': line_en,
                'predicted_translation': line_predicted
            })
    
    df = pd.DataFrame(data)
    return df

def translate_llm_pages(bubbles_input, prompt, client):
    #print("--------Text input: ---------\n", bubbles_input)
    question = prompt.format(text=bubbles_input)
    #print("--------Prompt: ---------\n", question)
    answer = ask_question(question, client)
    #print("--------Raw answer: ---------\n", answer)

    # Split the translations into sentences
    sentences = answer.split("\n")
    
    # Initialize an empty list to store the translated sentences
    translated_sentences = []
    
    # Iterate over each sentence
    for sentence in sentences:
        # Check if the sentence starts with a number followed by a dot and a space
        if sentence.strip() and sentence.split(". ", 1)[0].isdigit():
            # Remove the number and dot from the beginning of the sentence
            sentence = sentence.split(". ", 1)[1].strip()
        
            # Append the processed sentence to the list of translated sentences
            translated_sentences.append(sentence)
    
    # Join the translated sentences into a single string
    translated_text = "\n".join(translated_sentences)
    #print("--------Final answer: ---------")
    #print(translated_text)
    return translated_text

### Multimodal

#### Describe and translate

In [80]:
from itertools import zip_longest

def evaluate_model_multimodal_frames(client, test_dataset, prompt, description_prompt):
    data = []
    context = ""
    current_page = None
    frame_descriptions = []
    frame_texts = []

    for example in tqdm(test_dataset, desc="Evaluating"):
        frame_path = example['frame_path']
        page_number = int(os.path.basename(frame_path).split('_')[0])
        frame_number = int(os.path.basename(frame_path).split('_')[-1].split('.')[0])
        bubbles_ja = example['text_ja'].split('\n')
        bubbles_en = example['text_en'].split('\n')

        if current_page is None or current_page != page_number:
            if current_page is not None:
                print("Current page", current_page)
                # Process the accumulated frame descriptions and texts for the previous page
                translation_input = ""
                text_counter = 1
                for i, (description, text) in enumerate(zip(frame_descriptions, frame_texts)):
                    frame_info = f"Frame {i+1}:\n"
                    frame_info += f"Description: {description}\n"
                    frame_info += "Text: "
                    if text:
                        lines = text.split('\n')
                        for line in lines:
                            if line.strip():  # Skip empty lines
                                frame_info += f"{text_counter}. {line.split('. ', 1)[-1]}\n"
                                text_counter += 1
                    else:
                        frame_info += "None\n"
                    frame_info += "\n"
                    translation_input += frame_info

                predicted_translation = translate_llm_frames_multimodal(translation_input, prompt, client)
                predicted_lines = predicted_translation.split('\n')

                for line_ja, line_en, line_predicted in zip_longest(frame_texts, bubbles_en, predicted_lines, fillvalue=""):
                    data.append({
                        'frame_path': frame_path,
                        'bubble_ja': line_ja,
                        'bubble_en': line_en,
                        'predicted_translation': line_predicted
                    })

            current_page = page_number
            context = ""
            frame_descriptions = []
            frame_texts = []

        # Generate frame description
        question = description_prompt.format(frame_number=frame_number, context=context)
        #print(question)
        frame_description = process_image_llava(frame_path, question)
        frame_descriptions.append(frame_description)

        # Update context with the current frame description
        context += f"{frame_number}. {frame_description}\n"

        if not bubbles_ja or all(not bubble.strip() for bubble in bubbles_ja):
            bubbles_input = ""
        else:
            bubbles_input = '\n'.join([f"{i+1}. {bubble}" for i, bubble in enumerate(bubbles_ja)])
        frame_texts.append(bubbles_input)

    # Process the last page
    if frame_descriptions and frame_texts:
        translation_input = ""
        text_counter = 1
        for i, (description, text) in enumerate(zip(frame_descriptions, frame_texts)):
            frame_info = f"Frame {i+1}:\n"
            frame_info += f"Description: {description}\n"
            frame_info += "Text: "
            if text:
                lines = text.split('\n')
                for line in lines:
                    if line.strip():  # Skip empty lines
                        frame_info += f"{text_counter}. {line.split('. ', 1)[-1]}\n"
                        text_counter += 1
            else:
                frame_info += "None\n"
            frame_info += "\n"
            translation_input += frame_info

        predicted_translation = translate_llm_frames_multimodal(translation_input, prompt, client)
        predicted_lines = predicted_translation.split('\n')

        for line_ja, line_en, line_predicted in zip_longest(frame_texts, bubbles_en, predicted_lines, fillvalue=""):
            data.append({
                'frame_path': frame_path,
                'bubble_ja': line_ja,
                'bubble_en': line_en,
                'predicted_translation': line_predicted
            })

    df = pd.DataFrame(data)
    return df

def translate_llm_frames_multimodal(translation_input, prompt, client):
    #print("--------Text input: ---------\n", translation_input)
    question = prompt.format(input=translation_input)
    #print("--------Prompt: ---------\n", question)
    answer = ask_question(question, client,max_tokens=2000)
    #print("--------Raw answer: ---------\n", answer)

    # Split the translations into sentences
    sentences = answer.split("\n")
    
    # Initialize an empty list to store the translated sentences
    translated_sentences = []
    
    # Iterate over each sentence
    for sentence in sentences:
        # Check if the sentence starts with a number followed by a dot and a space
        if sentence.strip() and sentence.split(". ", 1)[0].isdigit():
            # Remove the number and dot from the beginning of the sentence
            sentence = sentence.split(". ", 1)[1].strip()
        
            # Append the processed sentence to the list of translated sentences
            translated_sentences.append(sentence)
    
    # Join the translated sentences into a single string
    translated_text = "\n".join(translated_sentences)
    print("--------Final answer: ---------")
    print(translated_text)
    return translated_text

#### Saving descriptions and then translating

In [ ]:
import json
from itertools import zip_longest
import pandas as pd
import os
from tqdm import tqdm

def evaluate_danbooru_frames_process_images(test_dataset):
    data = []
    current_page = None
    frame_descriptions = []
    frame_texts_ja = []
    frame_texts_en = []
    for example in tqdm(test_dataset, desc="Evaluating"):
        frame_path = example['frame_path']
        page_number = int(os.path.basename(frame_path).split('_')[0])
        frame_number = int(os.path.basename(frame_path).split('_')[-1].split('.')[0])
        bubbles_ja = example['text_ja'].split('\n')
        bubbles_en = example['text_en'].split('\n')
        if current_page is None or current_page != page_number:
            if current_page is not None:
                data.append({
                    'page_number': current_page,
                    'frame_descriptions': frame_descriptions,
                    'frame_texts_ja': frame_texts_ja,
                    'frame_texts_en': frame_texts_en
                })
            current_page = page_number
            frame_descriptions = []
            frame_texts_ja = []
            frame_texts_en = []
        # Read frame description from text file
        description_file = os.path.splitext(frame_path)[0] + '.txt'
        with open(description_file, 'r') as f:
            frame_description = f.read().strip()
        frame_descriptions.append(frame_description)
        if not bubbles_ja or all(not bubble.strip() for bubble in bubbles_ja):
            bubbles_input_ja = ""
        else:
            bubbles_input_ja = '\n'.join([f"{i+1}. {bubble}" for i, bubble in enumerate(bubbles_ja)])
        frame_texts_ja.append(bubbles_input_ja)
        if not bubbles_en or all(not bubble.strip() for bubble in bubbles_en):
            bubbles_input_en = ""
        else:
            bubbles_input_en = '\n'.join([f"{i+1}. {bubble}" for i, bubble in enumerate(bubbles_en)])
        frame_texts_en.append(bubbles_input_en)
    # Process the last page
    if frame_descriptions and frame_texts_ja and frame_texts_en:
        data.append({
            'page_number': current_page,
            'frame_descriptions': frame_descriptions,
            'frame_texts_ja': frame_texts_ja,
            'frame_texts_en': frame_texts_en
        })
    # Save the data to a JSON file
    with open("frame_data_mini.json", 'w') as f:
        json.dump(data, f)
    return data

def evaluate_model_multimodal_frames_process_images(test_dataset, description_prompt, mode):
    data = []
    context = ""
    current_page = None
    frame_descriptions = []
    frame_texts_ja = []
    frame_texts_en = []
    for example in tqdm(test_dataset, desc="Evaluating"):
        frame_path = example['frame_path']
        page_number = int(os.path.basename(frame_path).split('_')[0])
        frame_number = int(os.path.basename(frame_path).split('_')[-1].split('.')[0])
        bubbles_ja = example['text_ja'].split('\n')
        bubbles_en = example['text_en'].split('\n')
        if current_page is None or current_page != page_number:
            if current_page is not None:
                data.append({
                    'page_number': current_page,
                    'frame_descriptions': frame_descriptions,
                    'frame_texts_ja': frame_texts_ja,
                    'frame_texts_en': frame_texts_en
                })
            current_page = page_number
            context = ""
            frame_descriptions = []
            frame_texts_ja = []
            frame_texts_en = []
        # Generate frame description
        question = description_prompt.format(frame_number=frame_number, context=context)
        frame_description = process_image_llava(frame_path, question, mode)
        frame_descriptions.append(frame_description)
        # Update context with the current frame description
        context += f"{frame_number}. {frame_description}\n"
        if not bubbles_ja or all(not bubble.strip() for bubble in bubbles_ja):
            bubbles_input_ja = ""
        else:
            bubbles_input_ja = '\n'.join([f"{i+1}. {bubble}" for i, bubble in enumerate(bubbles_ja)])
        frame_texts_ja.append(bubbles_input_ja)
        if not bubbles_en or all(not bubble.strip() for bubble in bubbles_en):
            bubbles_input_en = ""
        else:
            bubbles_input_en = '\n'.join([f"{i+1}. {bubble}" for i, bubble in enumerate(bubbles_en)])
        frame_texts_en.append(bubbles_input_en)
    # Process the last page
    if frame_descriptions and frame_texts_ja and frame_texts_en:
        data.append({
            'page_number': current_page,
            'frame_descriptions': frame_descriptions,
            'frame_texts_ja': frame_texts_ja,
            'frame_texts_en': frame_texts_en
        })
    # Save the data to a JSON file
    with open("frame_data_"+mode+".json", 'w') as f:
        json.dump(data, f)
    return data

def evaluate_model_multimodal_frames_processed(client, prompt, mode):
    with open("frame_data_"+mode+".json", 'r') as f:
        data = json.load(f)
    processed_data = []
    for page_data in tqdm(data, desc="Processing pages"):
        translation_input = ""
        text_counter = 1
        for i, (description, text_ja, text_en) in enumerate(zip(page_data['frame_descriptions'], page_data['frame_texts_ja'], page_data['frame_texts_en'])):
            frame_info = f"Frame {i+1}:\n"
            frame_info += f"Description: {description}\n"
            frame_info += "Text: "
            if text_ja:
                lines_ja = text_ja.split('\n')
                lines_en = text_en.split('\n')
                for line_ja, line_en in zip(lines_ja, lines_en):
                    if line_ja.strip():  # Skip empty lines
                        frame_info += f"{text_counter}. {line_ja.split('. ', 1)[-1]}\n"
                        text_counter += 1
            else:
                frame_info += "None\n"
            frame_info += "\n"
            translation_input += frame_info
        predicted_translation = translate_llm_frames_multimodal(translation_input, prompt, client)
        predicted_lines = predicted_translation.split('\n')
        
        # Deprocess bubble_ja and bubble_en
        deprocessed_lines_ja = []
        deprocessed_lines_en = []
        for text_ja, text_en in zip(page_data['frame_texts_ja'], page_data['frame_texts_en']):
            lines_ja = text_ja.split('\n')
            lines_en = text_en.split('\n')
            for line_ja, line_en in zip(lines_ja, lines_en):
                if line_ja.strip():
                    deprocessed_lines_ja.append(line_ja.split('. ', 1)[-1])
                if line_en.strip():
                    deprocessed_lines_en.append(line_en.split('. ', 1)[-1])
        
        # Deprocess predicted translations
        deprocessed_predicted_lines = [line.strip('"') for line in predicted_lines if line.strip()]

        for line_ja, line_en, line_predicted in zip_longest(deprocessed_lines_ja, deprocessed_lines_en, deprocessed_predicted_lines, fillvalue=""):
            processed_data.append({
                'page_number': page_data['page_number'],
                'bubble_ja': line_ja,
                'bubble_en': line_en,
                'predicted_translation': line_predicted
            })
    df = pd.DataFrame(processed_data)
    return df

## Prompts

### Prompt: Unimodal Approach

#### Zero shot

In [ ]:
# Bubble dataset
prompt_text_zeroshot_bubbles = """You are a manga translator from Japanese to English. 
Some rules to remember:
- Maintaining the contents' accuracy is important, but since texts are from manga, we want to prioritize naturalness and ease of communication.
- Instead of translating word by word, try to translate the whole sentence or phrase at once.
- Number of translated sentences should be the same as the number of input sentences.
- Return translations without additional explanations, comments, notes or interaction. 
- You'll be penalized if you add any additional information besides the format provided or if you don't follow the format provided.

Here's the format.

Input:
1. Sentence in Japanese

Translation:
1. Translation in English

Input:
1. {text}

Translation:

"""

# Frame dataset
prompt_text_zeroshot_frames = """You are a manga translator from Japanese to English. 
Some rules to remember:
- Maintaining the contents' accuracy is important, but since texts are from manga, we want to prioritize naturalness and ease of communication.
- Instead of translating word by word, try to translate the whole sentence or phrase at once.
- Number of translated sentences should be the same as the number of input sentences.
- Return translations without additional explanations, comments, notes or interactions.
- You'll be penalized if you add any additional information besides the format provided or if you don't follow the format provided.

Here's the format. Each sentence corresponds to one text bubble in a manga frame. 
Have in mind that this is a sequence of dialogs in manga. 

Input:
1. Sentence 1 in Japanese
2. Sentence 2 in Japanese
3. Sentence 3 in Japanese

Translation:
1. Translation 1 in English
2. Translation 2 in English
3. Translation 3 in English

Input:
{text}

Translation:

"""

# Page dataset
prompt_text_zeroshot_pages = """You are a manga translator from Japanese to English. 
Some rules to remember:
- Maintaining the contents' accuracy is important, but since texts are from manga, we want to prioritize naturalness and ease of communication.
- Instead of translating word by word, try to translate the whole sentence or phrase at once.
- Number of translated sentences should be the same as the number of input sentences.
- Return translations without additional explanations, comments, notes or interactions. Simply use the format for translations.

Here's the format. Each sentence corresponds to one text bubble in a manga page. 
Have in mind that this is a sequence of dialogs in manga. 

Input:
1. Sentence 1 in Japanese
2. Sentence 2 in Japanese
3. Sentence 3 in Japanese

Translation:
1. Translation 1 in English
2. Translation 2 in English
3. Translation 3 in English

Input:
{text}

Translation:
"""

#### One shot

In [14]:
# Bubble dataset
prompt_text_oneshot_bubbles = """You are a manga translator from Japanese to English. 
Some rules to remember:
- Maintaining the contents' accuracy is important, but since texts are from manga, we want to prioritize naturalness and ease of communication.
- Instead of translating word by word, try to translate the whole sentence or phrase at once.
- Number of translated sentences should be the same as the number of input sentences.
- Return translations without additional explanations, comments, notes or interaction. 
- You'll be penalized if you add any additional information besides the format provided or if you don't follow the format provided.

Here's the format.

Input:
1. Sentence in Japanese

Translation:
1. Translation in English


Input:
1. いいって言ったじゃないですか

Translation:
1. I told you I'm okay!


Input:
1. {text}

Translation:

"""

# Frame dataset
prompt_text_oneshot_frames = """You are a manga translator from Japanese to English. 
Some rules to remember:
- Maintaining the contents' accuracy is important, but since texts are from manga, we want to prioritize naturalness and ease of communication.
- Instead of translating word by word, try to translate the whole sentence or phrase at once.
- Number of translated sentences should be the same as the number of input sentences.
- Return translations without additional explanations, comments, notes or interactions.
- You'll be penalized if you add any additional information besides the format provided or if you don't follow the format provided.

Here's the format. Each sentence corresponds to one text bubble in a manga frame. 
Have in mind that this is a sequence of dialogs in manga. 

Input:
1. Sentence 1 in Japanese
2. Sentence 2 in Japanese
3. Sentence 3 in Japanese

Translation:
1. Translation 1 in English
2. Translation 2 in English
3. Translation 3 in English


Input:
1. たっ助かった
2. シオラといったか
3. ウチに雇ってほしいそうじゃないか

Translation:
1. oh, thank god.
2. siora, was it?
3. i heard you wanted us to hire you?


Input:
{text}

Translation:

"""

# Page dataset
prompt_text_oneshot_pages = """You are a manga translator from Japanese to English. 
Some rules to remember:
- Maintaining the contents' accuracy is important, but since texts are from manga, we want to prioritize naturalness and ease of communication.
- Instead of translating word by word, try to translate the whole sentence or phrase at once.
- Number of translated sentences should be the same as the number of input sentences.
- Return translations without additional explanations, comments, notes or interactions. Simply use the format for translations.

Here's the format. Each sentence corresponds to one text bubble in a manga page. 
Have in mind that this is a sequence of dialogs in manga. 

Input:
1. Sentence 1 in Japanese
2. Sentence 2 in Japanese
3. Sentence 3 in Japanese

Translation:
1. Translation 1 in English
2. Translation 2 in English
3. Translation 3 in English


Input:
1. じゃあなんで
2. 父さんは死んだんだ!!
3. まっ待て
4. 撃たないでくれ
5. 死ね!
6. 死ねぇーーー!!

Translation:
1. then why?!
2. why did my father have to die!!
3. wait, wait.
4. don't shoot me.
5. die!
6. die!!!!


Input:
{text}

Translation:
"""

### Prompt: Multimodal Approach

#### Multimodal: Full Description (Zero-shot)

In [15]:
prompt_multimodal_vision_zeroshot_frames = """You are an assistant that describes manga frames, in order to help with frames context for translation 
considering the context from previous frames.
Some rules to remember:
- Analyze the current manga frame carefully and describe the essential elements of the scene, characters, and their actions. Ignore the text and do not describe it, as it will be provided separately jointed with your description.
- Use a brief and concise description. Avoid redundant information. For example "The image shows a black and white manga panel" as you're describing manga panels that is redundant.
- Consider the context from the previous frames to maintain continuity and coherence in your descriptions.
- Do not add any additional explanations or interactions beyond the frame description.
- You'll be penalized if you add any extra information besides the format provided or if you don't follow the format.

You're looking at frame #{frame_number}
Frame descriptions:
{context}
{frame_number}. 

Frame description:
"""

#### Multimodal: Reduced Description (Few-shot)

In [16]:
prompt_multimodal_vision_fewshot_frames = """You are an assistant that describes manga frames concisely to help with frame context for translation, considering the context from previous frames.

Some rules to remember:
- Analyze the current manga frame carefully and describe the characters, and their actions in a very concise way in 1-2 sentences. Do not try to describe text in the image, as it will be provided separately jointed with your description.
- Use a brief and concise description. Avoid redundant information. For example "The image shows a black and white manga panel" as you're describing manga panels that is redundant.
- Do not add any additional explanations or interactions beyond the frame description.
- You'll be penalized if you add any extra information besides the format provided or if you don't follow the format.

Examples of concise frame descriptions:

Frame 0:
A young girl with short hair sits alone on a swing in a park, looking sad and lost in thought.

Frame 1:
The girl notices a small puppy approaching her, wagging its tail. She looks surprised and curious.

Frame 2:
The girl kneels down to pet the puppy, a gentle smile forming on her face. The puppy seems friendly and eager for attention.

Frame 3:
The girl hugs the puppy close, her eyes closed and a content expression on her face, finding comfort in the puppy's presence.


You're looking at frame #{frame_number}

Current frame description:
"""

#### Text: Zero-shot

In [48]:
# Frame dataset
prompt_multimodal_text_zeroshot = """You are a manga translator from Japanese to English, considering the context from the manga frames.
Some rules to remember:
- Maintaining the contents' accuracy is important, but since texts are from manga, we want to prioritize naturalness and ease of communication.
- Instead of translating word by word, try to translate the whole sentence or phrase at once, taking into account the context provided by the frame descriptions.
- Number of translated sentences should be the same as the number of input sentences.
- Return translations without additional explanations, comments, notes, or interactions, or you'll be penalized.
Here's the format. Each sentence corresponds to one text bubble in a manga frame. 
Have in mind that this is a sequence of dialogs in manga, 
and the frame descriptions provide additional context for the translation.

Here's the format example. 
Input:
Frame 0:
Description: Description frame 0 
Text: 1. Text 1
      2. Text 2

Frame 1:
Description: Description frame 1
Text: 3. Text 3
      4. Text 4
      5. Text 5

Output:
Translation:
1. Translation text 1
2. Translation text 2
3. Translation text 3
4. Translation text 4
5. Translation text 5

Next input:
{input}
Output:
Your translation:
"""

#### Text: Few shot

In [70]:
prompt_multimodal_text_fewshot = """You are a manga translator from Japanese to English. 
Some rules to remember:
- Maintaining the contents' accuracy is important, but since texts are from manga, we want to prioritize naturalness and ease of communication.
- Instead of translating word by word, try to translate the whole sentence or phrase at once.
- Number of translated sentences should be the same as the number of input sentences.
- Return ONLY translations without additional explanations, comments, or descriptions, otherwise you will be penalized. Simply use the format for translations.

Here's the format example. Each sentence corresponds to one text bubble in a manga page. Have in mind that this is a sequence of dialogs in manga. 

Frame 0:
Description: Description frame 0 
Text: 1. Text 1
      2. Text 2

Frame 1:
Description: Description frame 1
Text: 3. Text 3
      4. Text 4
      5. Text 5

Translation:
1. Translation text 1
2. Translation text 2
3. Translation text 3
4. Translation text 4
5. Translation text 5

Input:
Frame 0:
Description: A young girl with short hair sits alone on a swing in a park, looking sad and lost in thought.
Text: 1. どうしたらいいのかな...
      2. 私には友達がいないの...
Frame 1:  
Description: The girl notices a small puppy approaching her, wagging its tail. She looks surprised and curious.
Text: 3. あら？
      4. 子犬ちゃん、どうしたの？
      5. 私に何か用？
Frame 2:
Description: The girl kneels down to pet the puppy, a gentle smile forming on her face. The puppy seems friendly and eager for attention.
Text: 6. わぁ、かわいい！
      7. いい子だね
      8. 私と一緒にいてくれるの？
Frame 3:
Description: The girl hugs the puppy close, her eyes closed and a content expression on her face, finding comfort in the puppy's presence.  
Text: 9. ありがとう
      10. あなたのおかげで、少し元気が出たよ
      11. これからは一緒だね

Output:
Translation:
1. What should I do...
2. I don't have any friends...
3. Oh?
4. Little puppy, what's the matter?
5. Do you need something from me?
6. Wow, so cute!
7. You're a good boy/girl, aren't you?
8. Will you stay with me?
9. Thank you
10. Thanks to you, I feel a bit better
11. From now on, we'll be together


Next Input:
{input}

Output:
Your translation:
"""

#### Text: Chain-of-Thought

In [81]:
prompt_multimodal_text_CoT = """You are a manga translator from Japanese to English.
Your task is to translate the provided Japanese text from manga panels into natural and communicative English, while considering the visual context described for each frame.
Some rules to remember:
- Maintaining the contents' accuracy is important, but since texts are from manga, we want to prioritize naturalness and ease of communication.
- Instead of translating word by word, try to translate the whole sentence or phrase at once.
- Number of translated sentences should be the same as the number of input sentences.
- Use the Chain of Thought approach to analyze the text in relation to the frame description, considering the characters, actions, and emotions. Do this for each text line.
- Always follow this Chain of Thought structure:

[Initial Thought]: Decide to use [Context Analysis] for each text line followed by its [Translation].
[Context Analysis]: Analyze the text line in the context of its frame description.
[Translation]: Provide the English translation for the text line based on the analysis.
[Final Thought]: After translating all lines, decide to use [Translation Summary].
[Translation Summary]: Provide the final translations in order.

- Return ONLY translations without additional explanations, comments, or descriptions, otherwise you will be penalized. Simply use the format for translations.

Here's the format example. Each sentence corresponds to one text bubble in a manga page. Have in mind that this is a sequence of dialogs in manga. 

Input:
Frame 0:
Description: A boy and a girl stand facing each other, both looking angry and ready to fight.
Text: 1. おい、何だよ！
2. あんたが先に始めたんでしょ！
Frame 1:
Description: The boy clenches his fists, looking even more furious. The girl points at him accusingly.
Text: 3. うるさい！お前なんかに負けないよ！
4. それはこっちのセリフよ！かかってきなさい！

Output:
[Initial Thought]: I will start by analyzing each text line using [Context Analysis] and then provide the [Translation].

Frame 0:
Text 1: おい、何だよ！
[Context Analysis]:
The boy seems to be initiating the confrontation.
He's using a rough, masculine way of speaking, showing his anger.
[Translation]: Hey, what's your problem?!

Text 2: あんたが先に始めたんでしょ！
[Context Analysis]:
The girl is responding to the boy's accusation.
She's using "あんた" which is a somewhat rude way to say "you", indicating her anger.
She's accusing him of starting the fight first.
[Translation]: You're the one who started it!

Frame 1:
Text 3: うるさい！お前なんかに負けないよ！
[Context Analysis]:
The boy is telling the girl to shut up, using the rude "うるさい".
He declares that he won't lose to her, showing his determination and pride.
[Translation]: Shut up! I won't lose to the likes of you!

Text 4: それはこっちのセリフよ！かかってきなさい！
[Context Analysis]:
The girl retorts that she's the one who should be saying that.
She challenges him to bring it on, showing she's not backing down.
[Translation]: That's my line! Bring it on!


[Final Thought]: Now that I have translated all the lines, I will provide the final translations using [Translation Summary].
There were 4 text lines as input so I will provide the 4 translations in order.

[Translation Summary]:
1. Hey, what's your problem?!
2. You're the one who started it!
3. Shut up! I won't lose to the likes of you!
4. That's my line! Bring it on!


Next Input:
{input}

Output:
Your thoughts:
"""

## Unimodal experiments

### Zero-shot

#### No context

In [60]:
dataset = dataset_bubble
#dataset = dataset_bubble[-5:]

# ollama running on local
client = OpenAI(base_url='http://localhost:11434/v1/', api_key='ollama')
df = evaluate_model_unimodal_bubbles(client, dataset, prompt_text_zeroshot_bubbles)
calculate_metrics(df['predicted_translation'].tolist(), df['bubble_en'].tolist())

Evaluating: 100%|██████████| 1592/1592 [05:05<00:00,  5.20it/s]


BERTScore: {'precision': 0.8739814758300781, 'recall': 0.8815203309059143, 'f1': 0.8774098753929138}
BLEU4: 0.03065420889586295
BLEU4 tokenized: 0.04936391476314992
METEOR Score: 0.24821512368413856


#### Frame-level context

In [88]:
dataset = dataset_frame
#dataset = dataset_frame[:5]

# ollama running on local
client = OpenAI(base_url='http://localhost:11434/v1/', api_key='ollama')
df = evaluate_model_unimodal_frames(client, dataset, prompt_text_zeroshot_frames)
calculate_metrics(df['predicted_translation'].tolist(), df['bubble_en'].tolist())

Evaluating: 100%|██████████| 1069/1069 [05:16<00:00,  3.38it/s]


BERTScore: {'precision': 0.8762075901031494, 'recall': 0.8849802613258362, 'f1': 0.8802476525306702}
BLEU4: 0.04123609137339335
BLEU4 tokenized: 0.061968004908894835
METEOR Score: 0.27479403246644574


#### Page-level context

In [90]:
dataset = dataset_page
#dataset = dataset_page[:5]

# ollama running on local
client = OpenAI(base_url='http://localhost:11434/v1/', api_key='ollama')
df = evaluate_model_unimodal_pages(client, dataset, prompt_text_zeroshot_pages)
calculate_metrics(df['predicted_translation'].tolist(), df['bubble_en'].tolist())

Evaluating: 100%|██████████| 214/214 [04:17<00:00,  1.20s/it]


BERTScore: {'precision': 0.8770270347595215, 'recall': 0.8853311538696289, 'f1': 0.8808497190475464}
BLEU4: 0.03925300925941725
BLEU4 tokenized: 0.06323765633349297
METEOR Score: 0.2875569702377938


### One-shot

#### No context

In [95]:
dataset = dataset_bubble
#dataset = dataset_bubble[:5]

# ollama running on local
client = OpenAI(base_url='http://localhost:11434/v1/', api_key='ollama')
df = evaluate_model_unimodal_bubbles(client, dataset, prompt_text_oneshot_bubbles)
calculate_metrics(df['predicted_translation'].tolist(), df['ground_truth'].tolist())

BERTScore: {'precision': 0.873628556728363, 'recall': 0.8853239417076111, 'f1': 0.8791025280952454}
BLEU4: 0.04166433795086157
BLEU4 tokenized: 0.06292865115282789
METEOR Score: 0.2643136623172951


#### Frame-level context

In [96]:
dataset = dataset_frame
#dataset = dataset_frame[:5]

# ollama running on local
client = OpenAI(base_url='http://localhost:11434/v1/', api_key='ollama')
df = evaluate_model_unimodal_frames(client, dataset, prompt_text_oneshot_frames)
calculate_metrics(df['predicted_translation'].tolist(), df['bubble_en'].tolist())

Evaluating: 100%|██████████| 1069/1069 [05:16<00:00,  3.38it/s]


BERTScore: {'precision': 0.8797049522399902, 'recall': 0.8872645497322083, 'f1': 0.8831325769424438}
BLEU4: 0.04216152638065262
BLEU4 tokenized: 0.06954673636935513
METEOR Score: 0.2781056645339773


#### Page-level context

In [97]:
dataset = dataset_page
#dataset = dataset_page[:5]

# ollama running on local
client = OpenAI(base_url='http://localhost:11434/v1/', api_key='ollama')
df = evaluate_model_unimodal_pages(client, dataset, prompt_text_oneshot_pages)
calculate_metrics(df['predicted_translation'].tolist(), df['bubble_en'].tolist())

Evaluating: 100%|██████████| 214/214 [04:16<00:00,  1.20s/it]


BERTScore: {'precision': 0.8720072507858276, 'recall': 0.8753203749656677, 'f1': 0.8732576370239258}
BLEU4: 0.04054144702489919
BLEU4 tokenized: 0.06264706485964747
METEOR Score: 0.23088064976954636


## Multimodal experiments

### Precompute image descriptions

#### PreCompute Full (Zero-Shot)

In [20]:
dataset = dataset_frame
#dataset = dataset_frame[9:]
evaluate_model_multimodal_frames_process_images(dataset, prompt_multimodal_vision_zeroshot_frames, mode="full")

Evaluating: 100%|██████████| 1069/1069 [1:09:27<00:00,  3.90s/it]


[{'page_number': 0,
  'frame_descriptions': [" The image shows a black and white manga panel featuring an animated character. This is the first frame of this particular scene, so there's no context from previous frames to consider here.\nThe central figure in the foreground appears to be male with dark hair tied back at his forehead; he has prominent eyebrows that are slightly furrowed as if concentrating or focused"],
  'frame_texts_ja': ['1. 綴じ眼のシオラ\n2. 朽鷹みつき'],
  'frame_texts_en': ['1. bound eye siora\n2. Mitsuki Kuchitaka']},
 {'page_number': 1,
  'frame_descriptions': [" The image shows a black and white manga panel with an interior setting, possibly within the confines of what appears to be a school or educational institution. There are multiple characters present in this frame: one character is seated at a desk on the left side; another person stands near that individual towards the center-left area of the image; there's also someone standing further",
   " The image shows a bla

#### PreCompute Reduced (Few-Shot)

In [ ]:
dataset = dataset_frame
#dataset = dataset_frame[5:8]
evaluate_model_multimodal_frames_process_images(dataset, prompt_multimodal_vision_fewshot_frames, mode="reduced")

#### PreCompute Minimal (Danbooru)

In [299]:
dataset = dataset_frame
#dataset = dataset_frame[5:8]
evaluate_danbooru_frames_process_images(dataset)

Evaluating: 100%|██████████| 1069/1069 [00:02<00:00, 382.16it/s]


[{'page_number': 0,
  'frame_descriptions': ['1girl, solo, gloves, weapon, fingerless gloves, gun, bandages, handgun'],
  'frame_texts_ja': ['1. 綴じ眼のシオラ\n2. 朽鷹みつき'],
  'frame_texts_en': ['1. bound eye siora\n2. Mitsuki Kuchitaka']},
 {'page_number': 1,
  'frame_descriptions': ['no humans, door',
   '1girl',
   '1girl, solo, long hair, breasts, open mouth, skirt, large breasts, pantyhose, belt, camisole, shouting',
   '1girl, long hair, 1boy, spiked hair'],
  'frame_texts_ja': ['',
   '1. だからっ',
   '1. 知らないって言ってるだろっ\n2. そんな借金なんて!',
   '1. そうは言ってもなぁ\n2. レーネ...'],
  'frame_texts_en': ['',
   "1. I'm telling you!!",
   "1. I don't know what you're talking about!\n2. i don't owe you!",
   "1. well, I'm sorry...\n2. lene..."]},
 {'page_number': 2,
  'frame_descriptions': ['1girl, 1boy, spiked hair',
   '1girl, solo, long hair, looking at viewer, smile, sweat, sweatdrop, grin',
   '1girl, feet, toes, sandals, toenails',
   '1girl, solo, weapon, sword, blunt bangs, fingerless gloves, scarf, he

### Zero-shot

#### Zero-shot with Full Description 

In [62]:
mode = "full"
client = OpenAI(base_url='http://localhost:11434/v1/', api_key='ollama')
df = evaluate_model_multimodal_frames_processed(client, prompt_multimodal_text_zeroshot, mode)
calculate_metrics(df['predicted_translation'].tolist(), df['bubble_en'].tolist())

Processing pages: 100%|██████████| 213/213 [04:31<00:00,  1.28s/it]


BERTScore: {'precision': 0.8756840825080872, 'recall': 0.8844903111457825, 'f1': 0.8797274231910706}
BLEU4: 0.0409367459981405
BLEU4 tokenized: 0.06299710206458116
METEOR Score: 0.2829111653416422


#### Zero-shot with Reduced Description 

In [63]:
mode = "reduced"
client = OpenAI(base_url='http://localhost:11434/v1/', api_key='ollama')
df = evaluate_model_multimodal_frames_processed(client, prompt_multimodal_text_zeroshot, mode)
calculate_metrics(df['predicted_translation'].tolist(), df['bubble_en'].tolist())

Processing pages: 100%|██████████| 213/213 [04:22<00:00,  1.23s/it]


BERTScore: {'precision': 0.8700782656669617, 'recall': 0.8780929446220398, 'f1': 0.873737096786499}
BLEU4: 0.038640635449264105
BLEU4 tokenized: 0.06030392866113665
METEOR Score: 0.2673135533320433


#### Zero-shot with Minimum Description 

In [64]:
mode = "mini"
df = evaluate_model_multimodal_frames_processed(client, prompt_multimodal_text_zeroshot, mode)
calculate_metrics(df['predicted_translation'].tolist(), df['bubble_en'].tolist())

Processing pages: 100%|██████████| 213/213 [04:14<00:00,  1.19s/it]


BERTScore: {'precision': 0.8730208873748779, 'recall': 0.8812922835350037, 'f1': 0.876798152923584}
BLEU4: 0.04092907766859722
BLEU4 tokenized: 0.06694333539102806
METEOR Score: 0.2838879379244458


### Few-shot

#### Few-shot with Full Description 

In [65]:
mode = "full"
client = OpenAI(base_url='http://localhost:11434/v1/', api_key='ollama')
df = evaluate_model_multimodal_frames_processed(client, prompt_multimodal_text_fewshot, mode)
calculate_metrics(df['predicted_translation'].tolist(), df['bubble_en'].tolist())

Processing pages: 100%|██████████| 213/213 [04:14<00:00,  1.19s/it]


BERTScore: {'precision': 0.8814326524734497, 'recall': 0.8883970975875854, 'f1': 0.884576678276062}
BLEU4: 0.049559238705096605
BLEU4 tokenized: 0.07727048975320763
METEOR Score: 0.2910176282682076


#### Few-shot with Reduced Description 

In [66]:
mode = "reduced"
client = OpenAI(base_url='http://localhost:11434/v1/', api_key='ollama')
df = evaluate_model_multimodal_frames_processed(client, prompt_multimodal_text_fewshot, mode)
calculate_metrics(df['predicted_translation'].tolist(), df['bubble_en'].tolist())

Processing pages: 100%|██████████| 213/213 [04:06<00:00,  1.16s/it]


BERTScore: {'precision': 0.8786497116088867, 'recall': 0.8868580460548401, 'f1': 0.8824148178100586}
BLEU4: 0.0418235977302566
BLEU4 tokenized: 0.0689566163239105
METEOR Score: 0.29005104335646614


#### Few-shot with Minimal Description 

In [67]:
mode = "mini"
df = evaluate_model_multimodal_frames_processed(client, prompt_multimodal_text_fewshot, mode)
calculate_metrics(df['predicted_translation'].tolist(), df['bubble_en'].tolist())

Processing pages: 100%|██████████| 213/213 [03:56<00:00,  1.11s/it]


BERTScore: {'precision': 0.8803597092628479, 'recall': 0.8884344696998596, 'f1': 0.884064793586731}
BLEU4: 0.05025843727973572
BLEU4 tokenized: 0.07720069192589714
METEOR Score: 0.2969986548705256


### Chain of Thought

#### CoT with Full Description 

In [84]:
mode = "full"
client = OpenAI(base_url='http://localhost:11434/v1/', api_key='ollama')
df = evaluate_model_multimodal_frames_processed(client, prompt_multimodal_text_CoT, mode)
calculate_metrics(df['predicted_translation'].tolist(), df['bubble_en'].tolist())

Processing pages:   0%|          | 1/213 [00:09<32:29,  9.19s/it]

--------Final answer: ---------
Shiora of the Knotted Eyes
On the Brink of Collapse


Processing pages:   1%|          | 2/213 [00:15<27:18,  7.76s/it]

--------Final answer: ---------
That's why!
They're saying they don't know, aren't they?!
Such debts are unthinkable!
Even if they say so...
(Rène...?)


Processing pages:   1%|▏         | 3/213 [00:24<28:27,  8.13s/it]

...

Processing pages: 100%|█████████▉| 212/213 [29:09<00:07,  7.42s/it]

--------Final answer: ---------
It's possible to get lost on a moonless night.
I'm here for you during those times.
I served under Akihito in a bygone era.
A thousand years ago.
The thoughts of Chitose.
Your judgment is what counts!


Processing pages: 100%|██████████| 213/213 [29:16<00:00,  8.25s/it]

--------Final answer: ---------
Ah, I see...
That's impossible!
Alright then...
Alright then, tomorrow is grave-cleaning day!
Oh, shut up...
Don't rush!


BERTScore: {'precision': 0.8717860579490662, 'recall': 0.8806539177894592, 'f1': 0.8758238554000854}
BLEU4: 0.031468230554528145
BLEU4 tokenized: 0.04986692360356542
METEOR Score: 0.2588438254863637


#### CoT with Reduced Description 

In [82]:
mode = "reduced"
client = OpenAI(base_url='http://localhost:11434/v1/', api_key='ollama')
df = evaluate_model_multimodal_frames_processed(client, prompt_multimodal_text_CoT, mode)
calculate_metrics(df['predicted_translation'].tolist(), df['bubble_en'].tolist())

Processing pages:   0%|          | 0/213 [00:00<?, ?it/s]

Processing pages:   0%|          | 1/213 [00:06<21:41,  6.14s/it]

--------Final answer: ---------
Siola, the one with bound eyes.
A hawk that's lost its way.


Processing pages:   1%|          | 2/213 [00:12<22:12,  6.31s/it]

--------Final answer: ---------
So that's why...
You're just saying you don't know?!
I'm not paying that kind of debt!
Yeah, right...
Leene...


...

Processing pages: 100%|█████████▉| 212/213 [28:06<00:07,  7.29s/it]

--------Final answer: ---------
In a moonless night, it's possible to stray off course.
That's why I'm here, for times like those.
I've served in Fuji since the Heian era.
A thousand years ago...
The thoughts of a thousand years...
That's your judgment!


Processing pages: 100%|██████████| 213/213 [28:12<00:00,  7.94s/it]

--------Final answer: ---------
Ah, I see...
There's no way...
Okay, then...
Alright, tomorrow's grave-cleaning duty!
Oh, come on...
Don't rush!


BERTScore: {'precision': 0.8677208423614502, 'recall': 0.878793478012085, 'f1': 0.8728718161582947}
BLEU4: 0.027457178220848432
BLEU4 tokenized: 0.047493638871736965
METEOR Score: 0.25789154442623813


#### CoT with Minimal Description 

In [83]:
mode = "mini"
df = evaluate_model_multimodal_frames_processed(client, prompt_multimodal_text_CoT, mode)
calculate_metrics(df['predicted_translation'].tolist(), df['bubble_en'].tolist())

Processing pages:   0%|          | 1/213 [00:09<32:02,  9.07s/it]

--------Final answer: ---------
Shiora's stitched gaze.
She seizes the rotten hawk.


Processing pages:   1%|          | 2/213 [00:14<24:57,  7.10s/it]

--------Final answer: ---------
So that's why!
You're saying you don't know what I'm talking about?!
That kind of debt is unacceptable!
Yeah, I guess you're right...
Lane...?!


Processing pages:   1%|▏         | 3/213 [00:26<32:52,  9.39s/it]

...

Processing pages: 100%|█████████▉| 212/213 [29:39<00:07,  7.66s/it]

--------Final answer: ---------
Even on a moonless night, there's a chance you might get lost.
That's why I'm here for you.
I have served for generations since the Heian period.
For over a thousand years.
The thoughts of a thousand years.
Your fiery judgment is upon you!


Processing pages: 100%|██████████| 213/213 [29:45<00:00,  8.38s/it]

--------Final answer: ---------
Ah, I get it...
I'll never be able to...
Okay...
Tomorrow's going to be a grave-cleaning day!
Oh no...
Don't rush!


BERTScore: {'precision': 0.8633180856704712, 'recall': 0.8732235431671143, 'f1': 0.8678756356239319}
BLEU4: 0.02813368016257617
BLEU4 tokenized: 0.0501659848967806
METEOR Score: 0.2536338915516394
